<img src="https://raw.githubusercontent.com/marcellyhc/CAP-423-4-Ciencia-Dados-Geoespaciais/main/INPE.png" width="1000">

### Disciplina: CAP-423-4 Ciência de Dados Geoespaciais
#### Professores: Gilberto Queiroz, Karine Ferreira, Lúbia Vinhas e Rafael Santos
#### Palestrante: Rolf Simões

#### Aluna: Marcelly Homem Coelho   
#### Contato: marcellyhc@gmail.com

**Documentação do Pacote SITS disponível em:** https://e-sensing.github.io/sitsbook/

-------------------------------------------------------------------------------

# Classificação de Áreas Queimadas com SITS

**Descrição:** Este Jupyter Nootebook apresenta uma aplicação de classificação de áreas queimadas no bioma Amazônia. Para isso, serão usadas amostras de ocorrências de queimadas no ano de 2018. 

A área de estudo compreende o estado brasileiro de Piauí. Para a implementação será usado o pacote, em linguegem de programação R, conhecido como SITS.
A **Figura 1** representa a distribuição das amostras de queimadas e não queimadas no mapa do Brasil.

<img src="https://raw.githubusercontent.com/marcellyhc/CAP-423-4-Ciencia-Dados-Geoespaciais/main/fig_mapa_br_com_poligonos.PNG" width="600">

**Figura 1: Representação da distribuição das amostras de queimadas e não queimadas.**



Inicialmente, foram aplicadas algumas manipulações em um Banco de Dados Geográfico. A finalidade é obter um conjunto de dados de entrada, sobre queimadas, que será usado como amostras. Os comandos para manipulação das informações, em linguagem SQL, estão disponíveis em: https://github.com/marcellyhc/CAP-423-4-Ciencia-Dados-Geoespaciais  

Em um segundo momento, foi desenvolvido em algoritmo, em linguagem de programação Python, que realiza um pré-processamento no conjunto de dados resultante das consultas SQL. O resultado desta etapa consiste em um arquivo, do tipo csv, chamado: CAP423_pontos_aleatorios_poligonos_queimados_e_nqueimados.csv. Este algoritmo também está disponível no mesmo repositório. 

-------------------------------

## Instalar o pacote TERRA

In [1]:
install.packages("terra")

## Importas as bibliotecas

In [2]:
library(sf)

In [3]:
library(terra)

In [4]:
# Verificar os pacotes instalados
sessionInfo()

## Instalar o pacote SITS

In [5]:
# Rodar o seguinte comando antes de carregar o sits
system("apt update && apt install ca-certificates")

In [6]:
#install sits from bundle
system("cp -u -R ../input/sits-bundle-0-14-0/sits-bundle/* /usr/local/lib/R/site-library/")

# install sits library
library(sits)
library(sitsdata)

In [7]:
# Caso der erro, atualizar o sits com:
# devtools::install_github("e-sensing/sits")

## Leitura do conjunto de amostras de queimadas e não queimadas

**Descrição:** O conjunto de dados consiste em 10 polígonos queimados e 10 polígonos não queimados. Para cada um dos polígonos, foram sorteados 15 pontos para representação das ocorrências de queimadas e não queimadas. O conjunto de dados contém os seguintes atributos:
- gid: identificador do ponto dentro do polígono;
- aqm_id: identificador do polígono;
- ano: ano referente a amostra;
- op: identificador da orbita_ponto do Landsat;
- data_atual: data de passagem do satelite - fomato yyyy-mm-dd;
- classe: resultado da classificação validada por um auditor;
- x: logitude correspondente ao centróide do polígono;
- y: latitude correspondente ao centróide do polígono;
- start_date: data de início da série temporal - fomato yyyy-mm-dd;
- end_date: data de fim da série temporal - fomato yyyy-mm-dd.

O formato do arquivo é do tivo csv. A **Figura 2** apresenta os polígonos queimados e não queimados, com destaque para os pontos, gerados de forma aleatória, dentro dos polígonos. 


<img src="https://raw.githubusercontent.com/marcellyhc/CAP-423-4-Ciencia-Dados-Geoespaciais/main/fig_poligonos_com_pontos_aleatorios.PNG" width="800">

**Figura 2: Representação dos polígonos queimados e não queimados, com destaque para os pontos aleatórios selecionados dentro dos polígonos.**

In [8]:
# Ler o conjunto de dados de entrada do arquivo csv
df_original <- read.csv("../input/inputdata3/CAP423_pontos_aleatorios_poligonos_queimados_e_nqueimados.csv")

In [9]:
# Exibir o dataframe original
df_original

In [10]:
# Fazer uma cópia do dataframe original
df <- rbind(df_original)

In [11]:
# Exibir o dataframe 
df

## Pré-processamento dos dados

Nesta etapa, foram realizados alguns pré-processamentos no conjunto de dados de ocorrências de queimadas e não queimadas. O objetivo é obter um *dataframe* pré-processado para, em um segundo momento, gerar as sérias temporais para as classes **queimada** e **não-queimada**.

In [12]:
# Excluir algumas colunas do dataframe
df$X <- NULL
df$aqm_id <- NULL
df$ano <- NULL
df$op <- NULL
df$data_atual <- NULL

In [13]:
# Exibir o dataframe
df

In [14]:
# Renomear as colunas do dataframe
names(df)[1:4] <- c("id", "label", "longitude", "latitude")

In [15]:
# Exibir o dataframe
df

In [16]:
# Ordenar as posições das colunas do dataframe
df <- df[c("id", "longitude", "latitude", "start_date", "end_date", "label")]

In [17]:
# Exibir o dataframe
df

In [18]:
# Salvar o dataframe pré-processado em um arquivo do tipo csv
write.csv(df,"./CAP423_pontos_aleatorios_poligonos_queimados_e_nqueimados_preprocessing.csv", row.names = FALSE)

## Criar um cubo de dados usando o serviço do BDC

In [19]:
# Definir uma sub-região para cobrir a área onde ficam as amostras
area_bbox <- c(lon_min = -46,
               lon_max = -44,
               lat_min = -10,
               lat_max = -7)

In [20]:
# Definir uma região menor (usá-la sits_classify() pata teste)
area_bbox_test <- c(lon_min = -45.5,
                    lon_max = -44.5,
                    lat_min = -9,
                    lat_max = -8)

In [21]:
# Inserir as credenciais disponibilizadas pelo BDC
Sys.setenv("BDC_ACCESS_KEY" = "")

In [22]:
# Listar as collections disponíveis no BDC
sits_list_collections(source = "BDC")

In [23]:
# Criar um cubo de dados usando 'LC8_30_16D_STK-1' para cobrir a área de estudo
# Imagens Landsat-8 sobre a região do Piauí

LC8_cube <- sits_cube(
    source = "BDC",                    # data is stored on BDC                   
    collection = "LC8_30_16D_STK-1",   # collection on BDC
    name = "my_cube",                  # Nome do cubo de dados
    bands = c("NDVI", "B4", "B5"),     # Bandas espectrais selecionadas
    roi = area_bbox,                   # Região de interesse
    start_date = "2018-01-01",         # Menor data do df
    end_date = "2018-12-31"            # Maior data do df
)

In [24]:
# Exibir os metadados do cubo criado
print(LC8_cube)

In [25]:
# Exibir a quantidade de observações do cubo 
sits_timeline(LC8_cube)

## Plotar as bandas espectrais disponíveis no cubo de dados

In [26]:
samples_series <- sits_get_data(
    cube = LC8_cube,
    file = './CAP423_pontos_aleatorios_poligonos_queimados_e_nqueimados_preprocessing.csv',
    bands = c("NDVI", "B4", "B5"),
    multicores = 4
)

In [27]:
# Mostras as amostras
print(samples_series)

In [28]:
# Exibir a quantidade de observações das amostras
sits_timeline(samples_series)

In [29]:
# Plotar as séries temporais das amostras
plot(samples_series)

**Observação:** Pode-se observar que o gráficos de séries temporais referente aos polígonos queimados do Indice de Vegetação de Diferença Normalizada (NDVI), apresentou uma redução no valor do índice próximo a data de queimada no mês de setembro (mais especificamente no dia 22 de setembro de 2018). 

In [30]:
samples_series$time_series

## Treinamento de um modelo de Aprendizado de Máquina

**Observação:** Depois de obter as séries temporais dos pontos dentro dos polígonos queimados e não queimados, a próxima etapa é selecionar um subconjunto para usar como exemplos de treinamento de um modelo de aprendizado de máquina. 

**Observação:** Inicialmente, será realizada a seleção de *features* para treinamento do modelo de aprendizado de máquina. As *features* selecionadas são as bandas espectrais: NDVI, RED e NIR. Estas correspondem as bandas NDVI, B4 e B5, respectivamente. 

In [31]:
# Gerar as amostras para treinamento do modelo
my_samples <- sits_select(
    samples_series, 
    bands = c("NDVI", "B4", "B5")
)

In [32]:
my_samples

In [33]:
# Treinar o modelo de Machine Learning
ml_model_01 <- sits_train(
    data = my_samples,
    ml_method = sits_rfor() # Random Forest
)

## Classificação do cubo de dados

**Observação:** A próxima etapa consiste em classificar o cubo de dados. 

O resultado da classificação será um conjunto de **Mapas de Probabilidades** para as classes queimada e não-queimada. Para cada mapa, o valor do pixel é proporcional à probabilidade de pertencer a classe. 

In [34]:
# Gerar o mapa de probabilidades para o modelo 
probs_cube <- sits_classify(
    data = LC8_cube,
    ml_model = ml_model_01,
    multicores = 4,
    memsize = 12, # Controlar o uso de memória
    outputdir = ".",
    verbose = TRUE,
    progress = TRUE,
    roi = area_bbox #area_bbox_test
)

In [35]:
# Visualizar o resultado plotando o mapa de probabilidades 
options(repr.plot.width = 18, repr.plot.height = 6)
plot(probs_cube)

## Gerar a matriz de confusão para dois modelos definidos

### Modelo: Random Forest

In [36]:
acc_rfor <- sits_kfold_validate(my_samples,
                                folds = 5,
                                multicores = 2,
                                ml_method = sits_rfor(num_trees = 2000)
)

In [37]:
print(acc_rfor)

**Discussão dos resultados:** No primeiro experimento, baseado no modelo *Random Forest*, buscou-se avaliar a acurácia da classificação de amostras de queimadas e não queimadas. A tabela acima apresenta a matriz de confusão que permite visualizar a dispersão dos valores previstos para as classes reais. 
Pode-se observar que apenas duas instâncias de classe real queimada foram classificadas erroneamente como não queimadas.

A partir da matriz de confusão é possível determinar a acurácia de um modelo de aprendizado de máquina. Para este experimento a acurácia foi de 99,33%.

### Modelo: XGBoost

In [38]:
acc_xgb <- sits_kfold_validate(my_samples,
                               folds = 5,
                               ml_method = sits_xgboost()
)

In [39]:
print(acc_xgb)

**Discussão dos resultados:** No segundo experimento, baseado no modelo *XGBoost*, novamente, buscou-se avaliar a acurácia da classificação de amostras de queimadas e não queimadas. A tabela acima apresenta a matriz de confusão. Pode-se observar que apenas duas instâncias de classe real queimada foram classificadas erroneamente como não queimadas e uma amostra de classe real não queimada foi classificada erroneamente como queimada.

A partir da matriz de confusão é possível determinar a acurácia de um modelo de aprendizado de máquina. Para este experimento a acurácia foi de 99%.

### Considerações finais
Os sistemas de classificação automática de áreas queimadas têm um enorme potencial de promover a redução do tempo necessário para realizar a definição de áreas queimadas anualmente. No contexto econômico, a implantação de sistemas de classificação automática traz a expectativa de converter o trabalhos de auditoria em tratamento de eventos duvidosos gerados por um classificador e, assim, minimizar custos e reduzir o tempo de classificação.

Esta atividade visa desenvolver um método de classificação de áreas queimadas por meio de séries temporais. Para isso, é usado o pacote SITS.

Analisando os resultados obtidos pelo modelos *Random Forest* e *XGBoost*, sugere-se que estes métodos sçao apropriados para o desenvolvimento de um sistema de classificação automática de áreas queimadas. Uma hipótese para esta conclusão é que, em ambos ps experimentos, o valor da acurácia foi superior ao valor estabelecido como ótimo (95%). Estes resultados mostram que é viável usar a estratégia para promover a classificação automática de áreas queimadas.